The idea:
<br> 
Implement Smith waterman
<br>
with blosum 62 and pam 250 as options for the scoring values 

import libraries <br> IntEnum to have the classes as enumerated values

In [ ]:
from enum import IntEnum
import numpy as np

Assigning the constants for the type of files to align

In [ ]:
class FASTA_TYPE(IntEnum):
  DNA = 1
  PROTEIN = 2

Assigning the constants for the scores

In [ ]:
class DNA_score(IntEnum):
  MATCH = 1
  MISMATCH = -1
  GAP = -1

Assigning the constant values for the traceback

In [ ]:
class Trace(IntEnum):
    STOP = 0
    LEFT = 1 
    UP = 2
    DIAGONAL = 3

Reading the FASTA files

In [ ]:
def fasta_reader(sequence_file):
    lines = open(sequence_file).readlines()
    #col 0 , row from 1 to finish to read all attributes 
    sequence_name_row = lines[0][1:]
    sequence = lines[1]
    #clean whitespaces
    return sequence_name_row.replace(" ","").strip(), sequence.strip()

Implement get Blosum 62 value

In [ ]:
def Blosum_value(seq1, seq2):
  blosum62 = {
"C":{"C":9, "S":-1, "T":-1, "P":-3, "A":0,  "G":-3,"N":-3,"D":-3, "E":-4, "Q":-3, "H":-3, "R":-3, "K":-3, "M":-1,"I":-1,"L":-1, "V":-1, "F":-2, "Y":-2, "W":-2},
"S":{"C":-1,"S":4,  "T":1,  "P":-1, "A":1,  "G":0, "N":1, "D":0,  "E":0,  "Q":0,  "H":-1, "R":-1, "K":0,  "M":-1,"I":-2,"L":-2, "V":-2, "F":-2, "Y":-2, "W":-3},
"T":{"C":-1,"S":1,  "T":4,  "P":1,  "A":-1, "G":1, "N":0, "D":1,  "E":0,  "Q":0,  "H":0,  "R":-1, "K":0,  "M":-1,"I":-2,"L":-2, "V":-2, "F":-2, "Y":-2, "W":-3},
"P":{"C":-3,"S":-1, "T":1,  "P":7,  "A":-1, "G":-2,"N":-1,"D":-1, "E":-1, "Q":-1, "H":-2, "R":-2, "K":-1, "M":-2,"I":-3,"L":-3, "V":-2, "F":-4, "Y":-3, "W":-4},
"A":{"C":0, "S":1,  "T":-1, "P":-1, "A":4,  "G":0, "N":-1,"D":-2, "E":-1, "Q":-1, "H":-2, "R":-1, "K":-1, "M":-1,"I":-1,"L":-1, "V":-2, "F":-2, "Y":-2, "W":-3},
"G":{"C":-3,"S":0,  "T":1,  "P":-2, "A":0,  "G":6, "N":-2,"D":-1, "E":-2, "Q":-2, "H":-2, "R":-2, "K":-2, "M":-3,"I":-4,"L":-4, "V":0,  "F":-3, "Y":-3, "W":-2},
"N":{"C":-3,"S":1,  "T":0,  "P":-2, "A":-2, "G":0, "N":6, "D":1,  "E":0,  "Q":0,  "H":-1, "R":0,  "K":0,  "M":-2,"I":-3,"L":-3, "V":-3, "F":-3, "Y":-2, "W":-4},
"D":{"C":-3,"S":0,  "T":1,  "P":-1, "A":-2, "G":-1,"N":1, "D":6,  "E":2,  "Q":0,  "H":-1, "R":-2, "K":-1, "M":-3,"I":-3,"L":-4, "V":-3, "F":-3, "Y":-3, "W":-4},
"E":{"C":-4,"S":0,  "T":0,  "P":-1, "A":-1, "G":-2,"N":0, "D":2,  "E":5,  "Q":2,  "H":0,  "R":0,  "K":1,  "M":-2,"I":-3,"L":-3, "V":-3, "F":-3, "Y":-2, "W":-3},
"Q":{"C":-3,"S":0,  "T":0,  "P":-1, "A":-1, "G":-2,"N":0, "D":0,  "E":2,  "Q":5,  "H":0,  "R":1,  "K":1,  "M":0, "I":-3,"L":-2, "V":-2, "F":-3, "Y":-1, "W":-2},
"H":{"C":-3,"S":-1, "T":0,  "P":-2, "A":-2, "G":-2,"N":1, "D":1,  "E":0,  "Q":0,  "H":8,  "R":0,  "K":-1, "M":-2,"I":-3,"L":-3, "V":-2, "F":-1, "Y":2,  "W":-2},
"R":{"C":-3,"S":-1, "T":-1, "P":-2, "A":-1, "G":-2,"N":0, "D":-2, "E":0,  "Q":1,  "H":0,  "R":5,  "K":2,  "M":-1,"I":-3,"L":-2, "V":-3, "F":-3, "Y":-2, "W":-3},
"K":{"C":-3,"S":0,  "T":0,  "P":-1, "A":-1, "G":-2,"N":0, "D":-1, "E":1,  "Q":1,  "H":-1, "R":2,  "K":5,  "M":-1,"I":-3,"L":-2, "V":-3, "F":-3, "Y":-2, "W":-3},
"M":{"C":-1,"S":-1, "T":-1, "P":-2, "A":-1, "G":-3,"N":-2,"D":-3, "E":-2, "Q":0,  "H":-2, "R":-1, "K":-1, "M":5, "I":1, "L":2,  "V":-2, "F":0,  "Y":-1, "W":-1},
"I":{"C":-1,"S":-2, "T":-2, "P":-3, "A":-1, "G":-4,"N":-3,"D":-3, "E":-3, "Q":-3, "H":-3, "R":-3, "K":-3, "M":1, "I":4, "L":2,  "V":1,  "F":0,  "Y":-1, "W":-3},
"L":{"C":-1,"S":-2, "T":-2, "P":-3, "A":-1, "G":-4,"N":-3,"D":-4, "E":-3, "Q":-2, "H":-3, "R":-2, "K":-2, "M":2, "I":2, "L":4,  "V":3,  "F":0,  "Y":-1, "W":-2},
"V":{"C":-1,"S":-2, "T":-2, "P":-2, "A":0,  "G":-3,"N":-3,"D":-3, "E":-2, "Q":-2, "H":-3, "R":-3, "K":-2, "M":1, "I":3, "L":1,  "V":4,  "F":-1, "Y":-1, "W":-3},
"F":{"C":-2,"S":-2, "T":-2, "P":-4, "A":-2, "G":-3,"N":-3,"D":-3, "E":-3, "Q":-3, "H":-1, "R":-3, "K":-3, "M":0, "I":0, "L":0,  "V":-1, "F":6,  "Y":3,  "W":1},
"Y":{"C":-2,"S":-2, "T":-2, "P":-3, "A":-2, "G":-3,"N":-2,"D":-3, "E":-2, "Q":-1, "H":2,  "R":-2, "K":-2, "M":-1,"I":-1,"L":-1, "V":-1, "F":3,  "Y":7,  "W":2},
"W":{"C":-2,"S":-3, "T":-3, "P":-4, "A":-3, "G":-2,"N":-4,"D":-4, "E":-3, "Q":-2, "H":-2, "R":-3, "K":-3, "M":-1,"I":-3,"L":-2, "V":-3, "F":1,  "Y":2,  "W":11}
}

  for s1 in blosum62:
    if s1 == seq1:
      for s2 in blosum62[s1]:
        if s2 == seq2:
          return blosum62[s1][s2]


In [ ]:
def Pam_value(seq1, seq2):
  Pam250 = {
"C":{"C":12, "S": 0, "T":-2, "P":-3, "A":-2, "G":-3, "N":-4,"D":-5, "E":-5, "Q":-5, "H":-3, "R":-4, "K":-5, "M":-5,"I":-2,"L":-6, "V":-2, "F":-4,  "Y": 0, "W":-8},
"S":{"C": 0, "S": 2, "T": 1, "P": 1, "A": 1, "G": 1, "N": 1,"D": 0, "E": 0, "Q":-1, "H":-1, "R": 0, "K": 0, "M":-2,"I":-1,"L":-3, "V":-1, "F":-3,  "Y":-3, "W":-2},
"T":{"C":-2, "S": 1, "T": 3, "P": 0, "A": 1, "G": 0, "N": 0,"D": 0, "E": 0, "Q":-1, "H":-1, "R":-1, "K": 0, "M":-1,"I": 0,"L":-2, "V": 0, "F":-3,  "Y":-3, "W":-5},
"P":{"C":-3, "S": 1, "T": 0, "P": 6, "A": 1, "G":-1, "N":-1,"D":-1, "E":-1, "Q": 0, "H": 0, "R": 0, "K":-1, "M":-2,"I":-2,"L":-3, "V":-1, "F":-5,  "Y":-5, "W":-6},
"A":{"C":-2, "S": 1, "T": 1, "P": 1, "A": 2, "G": 1, "N": 0,"D": 0, "E": 0, "Q": 0, "H":-1, "R":-2, "K":-1, "M":-1,"I":-1,"L":-2, "V": 0, "F":-4,  "Y":-3, "W":-6},
"G":{"C":-3, "S": 1, "T": 0, "P":-1, "A": 1, "G": 5, "N": 0,"D": 1, "E": 0, "Q":-1, "H":-2, "R":-3, "K":-2, "M":-3,"I":-2,"L":-4, "V":-1, "F":-5,  "Y":-5, "W":-7},
"N":{"C":-4, "S": 1, "T": 0, "P":-1, "A": 0, "G": 0, "N": 2,"D": 2, "E": 1, "Q": 1, "H": 2, "R": 0, "K": 1, "M":-2,"I":-2,"L":-3, "V":-2, "F":-4,  "Y":-2, "W":-4},
"D":{"C":-5, "S": 0, "T": 0, "P":-1, "A": 0, "G": 1, "N": 2,"D": 4, "E": 3, "Q": 2, "H": 1, "R":-1, "K": 0, "M":-3,"I":-2,"L":-4, "V":-2, "F":-6,  "Y":-4, "W":-7},
"E":{"C":-5, "S": 0, "T": 0, "P":-1, "A": 0, "G": 0, "N": 1,"D": 3, "E": 4, "Q": 2, "H": 1, "R":-1, "K": 0, "M":-2,"I":-2,"L":-3, "V":-2, "F":-5,  "Y":-4, "W":-7},
"Q":{"C":-5, "S":-1, "T":-1, "P": 0, "A": 0, "G":-1, "N": 1,"D": 2, "E": 2, "Q": 4, "H": 3, "R": 1, "K": 1, "M":-1,"I":-2,"L":-2, "V":-2, "F":-5,  "Y":-4, "W":-5},
"H":{"C":-3, "S":-1, "T":-1, "P": 0, "A":-1, "G":-2, "N": 2,"D": 1, "E": 1, "Q": 3, "H": 6, "R": 2, "K": 0, "M":-2,"I":-2,"L":-2, "V":-2, "F":-2,  "Y": 0, "W":-3},
"R":{"C":-4, "S": 0, "T":-1, "P": 0, "A":-2, "G":-3, "N": 0,"D":-1, "E":-1, "Q": 1, "H": 2, "R": 6, "K": 3, "M": 0,"I":-2,"L":-3, "V":-2, "F":-4,  "Y":-4, "W": 2},
"K":{"C":-5, "S": 0, "T": 0, "P":-1, "A":-1, "G":-2, "N": 1,"D": 0, "E": 0, "Q": 1, "H": 0, "R": 3, "K": 5, "M": 0,"I":-2,"L":-3, "V":-2, "F":-5,  "Y":-4, "W":-3},
"M":{"C":-5, "S":-2, "T":-1, "P":-2, "A":-1, "G":-3, "N":-2,"D":-3, "E":-2, "Q":-1, "H":-2, "R": 0, "K": 0, "M": 6, "I":2,"L": 4, "V": 2, "F": 0,  "Y":-2, "W":-4},
"I":{"C":-2, "S":-1, "T": 0, "P":-2, "A":-1, "G":-3, "N":-2,"D":-2, "E":-2, "Q":-2, "H":-2, "R":-2, "K":-2, "M": 2, "I":5,"L": 2, "V": 4, "F": 1,  "Y":-1, "W":-5},
"L":{"C":-6, "S":-3, "T":-2, "P":-3, "A":-2, "G":-4, "N":-3,"D":-4, "E":-3, "Q":-2, "H":-2, "R":-3, "K":-3, "M": 4, "I":2,"L": 6, "V": 2, "F": 2,  "Y":-1, "W":-2},
"V":{"C":-2, "S":-1, "T": 0, "P":-1, "A": 0, "G":-1, "N":-2,"D":-2, "E":-2, "Q":-2, "H":-2, "R":-2, "K":-2, "M": 2, "I":4,"L": 2, "V": 4, "F":-1,  "Y":-2, "W":-6},
"F":{"C":-4, "S":-3, "T":-3, "P":-5, "A":-4, "G":-5, "N":-4,"D":-6, "E":-5, "Q":-5, "H":-2, "R":-4, "K":-5, "M": 0, "I":1,"L": 2, "V":-1, "F": 9,  "Y": 7, "W": 0},
"Y":{"C": 0, "S":-3, "T":-3, "P":-5, "A":-3, "G":-5, "N":-2,"D":-4, "E":-4, "Q":-4, "H": 0, "R":-4, "K":-4, "M":-2,"I":-1,"L":-1, "V":-2, "F": 7,  "Y":10, "W": 0},
"W":{"C":-8, "S":-2, "T":-5, "P":-6, "A":-6, "G":-7, "N":-4,"D":-7, "E":-7, "Q":-5, "H":-3, "R":-2, "K":-3, "M":-4,"I":-5,"L":-2, "V":-6, "F": 0,  "Y": 0, "W":17}
}

  for s1 in Pam250:
    if s1 == seq1:
      for s2 in Pam250[s1]:
        if s2 == seq2:
          return Pam250[s1][s2]


Implementing the Smith Waterman Algorithm 

In [ ]:
def smith_waterman(seq1, seq2 , fasta_type, protein_score):
    # Generating the empty matrices for storing scores and tracing
    row = len(seq1) + 1
    col = len(seq2) + 1
    matrix = np.zeros(shape=(row, col), dtype=np.intc)  
    tracing_matrix = np.zeros(shape=(row, col), dtype=np.intc)  
    
    
    # Initialising the variables to find the highest scoring cell
    max_score = -1
    max_index = (-1, -1)
    
    # Calculating the scores for all cells in the matrix
    for i in range(1, row):
        for j in range(1, col):
          # FInding what type of FASTA File 
          if fasta_type == FASTA_TYPE.DNA:
            # Calculating the diagonal score (match score) //note to self : seq1-1 is because you added 1 to the row value and iterate on it but seq1 is still the same so you need to -1 
            
            match_value = DNA_score.MATCH if seq1[i - 1] == seq2[j - 1] else DNA_score.MISMATCH
            diagonal_score = matrix[i - 1, j - 1] + match_value
            
            # Calculating the vertical gap score
            vertical_score = matrix[i - 1, j] + DNA_score.GAP
            
            # Calculating the horizontal gap score
            horizontal_score = matrix[i, j - 1] + DNA_score.GAP
            
            # Taking the highest score 
            matrix[i, j] = max(0, diagonal_score, vertical_score, horizontal_score)

            # Protein alignment using blosum 62
          elif fasta_type == FASTA_TYPE.PROTEIN:
            # Calculating the diagonal score (match score)
            if protein_score == 1:
              match_value = Blosum_value(seq1[i-1], seq2[j-1] )
            elif protein_score == 2:
              match_value = Pam_value(seq1[i-1], seq2[j-1] )
            diagonal_score = matrix[i - 1, j - 1] + match_value
            
            # Calculating the vertical gap score
            vertical_score = matrix[i - 1, j] + DNA_score.GAP
            
            # Calculating the horizontal gap score
            horizontal_score = matrix[i, j - 1] + DNA_score.GAP
            
            # Taking the highest score 
            matrix[i, j] = max(0, diagonal_score, vertical_score, horizontal_score)
            
          # Tracking where the cell's value is coming from    
          if matrix[i, j] == 0: 
              tracing_matrix[i, j] = Trace.STOP
              
          elif matrix[i, j] == horizontal_score: 
              tracing_matrix[i, j] = Trace.LEFT
              
          elif matrix[i, j] == vertical_score: 
              tracing_matrix[i, j] = Trace.UP
              
          elif matrix[i, j] == diagonal_score: 
              tracing_matrix[i, j] = Trace.DIAGONAL 
              
          # Tracking the cell with the maximum score
          if matrix[i, j] >= max_score:
              max_index = (i,j)
              max_score = matrix[i, j]
                
    
    # Initialising the variables for tracing
    aligned_seq1 = ""
    aligned_seq2 = ""   
    current_aligned_seq1 = ""   
    current_aligned_seq2 = ""  
    (max_i, max_j) = max_index
    
    # Tracing and computing the pathway with the local alignment
    while tracing_matrix[max_i, max_j] != Trace.STOP:
      #diagonal move from both letters
        if tracing_matrix[max_i, max_j] == Trace.DIAGONAL:
            current_aligned_seq1 = seq1[max_i - 1]
            current_aligned_seq2 = seq2[max_j - 1]
            max_i = max_i - 1
            max_j = max_j - 1
      #insert a gap seq2
        elif tracing_matrix[max_i, max_j] == Trace.UP:
            current_aligned_seq1 = seq1[max_i - 1]
            current_aligned_seq2 = '-'
            max_i = max_i - 1    
      #insert a gap seq1
        elif tracing_matrix[max_i, max_j] == Trace.LEFT:
            current_aligned_seq1 = '-'
            current_aligned_seq2 = seq2[max_j - 1]
            max_j = max_j - 1
      #appened the current movement to string  
        aligned_seq1 = aligned_seq1 + current_aligned_seq1
        aligned_seq2 = aligned_seq2 + current_aligned_seq2
    
    # Reversing the order of the sequences
    aligned_seq1 = aligned_seq1[::-1]
    aligned_seq2 = aligned_seq2[::-1]
    #
    print(matrix)
    return aligned_seq1, aligned_seq2 ,max_score

In [ ]:
file_1_name, file_1 = fasta_reader("/content/drive/MyDrive/bioinformatics project /rand_prot1.fasta")
file_2_name, file_2  = fasta_reader("/content/drive/MyDrive/bioinformatics project /rand_prot2.fasta")
    
# Executing the Smith Waterman local alignment algorithm
# File 1, File 2, fasta type (1 -> DNA, 2 -> protein), protein score(1-> blosum 2->pam)
output_1, output_2, max_score = smith_waterman(file_1, file_2 , 2,1)

print('Maximum score of this local alignment: ' + str(max_score))   
print(file_1_name +' :'+ output_1  + '\n' + file_2_name +' :'+ output_2)

[[ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  9  8  7  6  5  4  3  2  1  0]
 [ 0  8 10  9  8  7  6  5  4  3  2]
 [ 0  7 12 11 10  9  8  7  6  5  4]
 [ 0  6 11 18 17 16 15 14 13 12 11]
 [ 0  5 10 17 25 24 23 22 21 20 19]
 [ 0  4  9 16 24 24 23 22 22 21 20]
 [ 0  3  8 15 23 23 22 21 21 26 25]
 [ 0  2  7 14 22 31 30 29 28 27 34]]
Maximum score of this local alignment: 34
Seq1 :CSADP---SIH
Seq2 :C-ADPHRREIH
